In [ ]:
import plotly
import plotly.graph_objects as go
import urllib, json
import pandas as pd
import psycopg2
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()
conn = psycopg2.connect(f"dbname={os.getenv('SQL_DATABASE')} user={os.getenv('SQL_DATABASE')} password={os.getenv('SQL_PASSWORD')} host={os.getenv('SQL_HOST')} port={os.getenv('SQL_PORT')}")
from sqlalchemy import create_engine

engine = create_engine(f"postgresql+psycopg2://{os.getenv('SQL_USER')}:{os.getenv('SQL_PASSWORD')}@{os.getenv('SQL_HOST')}:{os.getenv('SQL_PORT')}/{os.getenv('SQL_DATABASE')}")


In [ ]:
df_worflow_data = pd.read_sql("select * from analytics.flow_business_status_30169_ze_2024_v1", con=conn)
df_worflow_data['transition_code'] = df_worflow_data['source_step'] + '_' + df_worflow_data['target_step']
df_worflow_data_import = df_worflow_data[df_worflow_data.process=='import']
df_worflow_data_app = df_worflow_data[df_worflow_data.process=='app']

In [ ]:
df_worflow_data[~df_worflow_data.detection_id.isin(df_worflow_data_app)]


df_worflow_data[df_worflow_data.detection_id==6745499]

In [ ]:
df_states_data = pd.read_sql("select * from analytics.workflow_steps", con=conn)
df_states_data['color'] = df_states_data['step_code'].apply(lambda x : f"rgba({np.random.choice(range(256))}, {np.random.choice(range(256))}, {np.random.choice(range(256))}, 0.8)")
df_states_data['id'] = df_states_data.index.tolist()
df_states_data

In [ ]:
df_transitions_data = pd.read_sql("select * from analytics.workflow_transitions", con=conn)
df_transitions_data['transition_code'] = df_transitions_data['from_step']  + '_' + df_transitions_data['to_step']
df_transitions_data

In [ ]:
df_unknown_transitions = df_worflow_data[~df_worflow_data.transition_code.isin(df_transitions_data.transition_code.tolist())]
df_unknown_transitions.groupby(['transition_code','old_status','new_status']).size().reset_index()

In [ ]:
## 1. Données gen 

In [ ]:
# filter authorized transitions : 
def format_to_sankey_data(df_worflow_data,df_states_data, df_transitions_data, force_transitions=True):
    if force_transitions:
        df_worflow_data = df_worflow_data[df_worflow_data.transition_code.isin(df_transitions_data.transition_code.tolist())]
    flow_status = df_worflow_data.groupby(['transition_code','source_step','target_step']).size().reset_index()
    flow_status.rename(columns={0:'nb_transitions'}, inplace=True)
    flow_status['color'] = flow_status['transition_code'].apply(lambda x : f"rgba({np.random.choice(range(256))}, {np.random.choice(range(256))}, {np.random.choice(range(256))}, 0.8)")
    flow_status = flow_status.merge(df_states_data[['id','step_code']],how='left', left_on='target_step', right_on='step_code').rename(columns={'id':'target'}).drop(columns=['step_code'])
    flow_status = flow_status.merge(df_states_data[['id','step_code']],how='left', left_on='source_step', right_on='step_code').rename(columns={'id':'source'}).drop(columns=['step_code'])
    flow_status = flow_status[~(flow_status.target.isna())]
    return flow_status

In [ ]:
flow_status = format_to_sankey_data(df_worflow_data, df_states_data, df_transitions_data,  force_transitions=False) # full workflow
#flow_status = format_to_sankey_data(df_worflow_data_app,df_states_data,df_transitions_data,  force_transitions=False) # app workflow
#flow_status = format_to_sankey_data(df_worflow_data_import, df_states_data, df_transitions_data,  force_transitions=False) #  import workflow

In [ ]:
import plotly.io as pio

pio.renderers.default = "browser"

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity


fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  df_states_data['step_label'].tolist(),
      color =  df_states_data['color'].tolist()
    ),
    # Add links
    link = dict(
      source =  flow_status.source.tolist(),
      target =  flow_status.target.tolist(),
      value =  flow_status.nb_transitions.tolist(),
      label =  flow_status.transition_code.tolist(),
      color =  flow_status.color.tolist()
))])

fig.update_layout(title_text="Workflow des changements etats Aigle Milhaud 2024 ",
                  font_size=14)
fig.show(height= 1200, width= 1600)

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "browser"

# Funnel stages (example)
stages = [
    "Detected (Step 1)",
    "Suspect (2A)",
    "Prescribed (3B)",
    "Field Control (5)",
    "Official Report (6A)",
    "Rehabilitated (8)"
]

# Example counts for each stage
values = [1200, 850, 430, 200, 90, 45]

# Create funnel
fig = go.Figure(
    go.Funnel(
        y=stages,
        x=values,
        textinfo="value+percent initial+percent previous",
        marker={"color": ["#4c78a8", "#9ecae4", "#f28e2b", "#ffbe7d", "#59a14f", "#8cd17d"]},
        orientation='v'
    )
)

fig.update_layout(
    title="Detection Lifecycle Funnel (Example)",
    funnelmode="stack"
)

fig.show()

In [ ]:
import plotly.express as px
fig = px.funnel_area(names=["The 1st","The 2nd", "The 3rd", "The 4th", "The 5th"],
                    values=[5, 4, 3, 2, 1])
fig.show()